<a href="https://colab.research.google.com/github/anubhavgupta1/Udacity/blob/main/Frameworks/Tensorflow/v%201.0/RNNs/Char%20RNN/anna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


!mkdir -p drive
!google-drive-ocamlfuse drive

import os
os.chdir("drive/Udacity/RNN/")
!ls

sample_data
E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 146364 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.24-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.24-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.24-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0

In [2]:
!pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 44kB/s 
     |████████████████████████████████| 51kB 4.5MB/s 
     |████████████████████████████████| 3.8MB 42.9MB/s 
     |████████████████████████████████| 512kB 53.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=79fc42c6f8c7476187d054dd9cfb69638845cab38d7932a635ecf2b59ecd52e1
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninstalled tensorboard-2.4.0
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [3]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

In [4]:
with open('Anna Karenina.txt', 'r') as f:
    text=f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [5]:
print(text[:101])
print(encoded[:101])
print(len(vocab))

Chapter 1


Happy families are all alike; every unhappy family is unhappy in its own
way.

Everything
[31 64 57 72 76 61 74  1 16  0  0  0 36 57 72 72 81  1 62 57 69 65 68 65
 61 75  1 57 74 61  1 57 68 68  1 57 68 65 67 61 26  1 61 78 61 74 81  1
 77 70 64 57 72 72 81  1 62 57 69 65 68 81  1 65 75  1 77 70 64 57 72 72
 81  1 65 70  1 65 76 75  1 71 79 70  0 79 57 81 13  0  0 33 78 61 74 81
 76 64 65 70 63]
83


## Making training mini-batches

In [6]:
def get_batches(arr, batch_size, n_steps):
    '''Create a generator that returns batches of size
       batch_size x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the number of characters per batch and number of batches we can make
    chars_per_batch = batch_size * n_steps
    n_batches = len(arr)//chars_per_batch
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * chars_per_batch]
    
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y_temp = arr[:, n+1:n+n_steps+1]
        
        # For the very last batch, y will be one character short at the end of 
        # the sequences which breaks things. To get around this, I'll make an 
        # array of the appropriate size first, of all zeros, then add the targets.
        # This will introduce a small artifact in the last batch, but it won't matter.
        y = np.zeros(x.shape, dtype=x.dtype)
        y[:,:y_temp.shape[1]] = y_temp
        
        yield x, y

In [7]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[31 64 57 72 76 61 74  1 16  0]
 [ 1 57 69  1 70 71 76  1 63 71]
 [78 65 70 13  0  0  3 53 61 75]
 [70  1 60 77 74 65 70 63  1 64]
 [ 1 65 76  1 65 75 11  1 75 65]
 [ 1 37 76  1 79 57 75  0 71 70]
 [64 61 70  1 59 71 69 61  1 62]
 [26  1 58 77 76  1 70 71 79  1]
 [76  1 65 75 70  7 76 13  1 48]
 [ 1 75 57 65 60  1 76 71  1 64]]

y
 [[64 57 72 76 61 74  1 16  0  0]
 [57 69  1 70 71 76  1 63 71 65]
 [65 70 13  0  0  3 53 61 75 11]
 [ 1 60 77 74 65 70 63  1 64 65]
 [65 76  1 65 75 11  1 75 65 74]
 [37 76  1 79 57 75  0 71 70 68]
 [61 70  1 59 71 69 61  1 62 71]
 [ 1 58 77 76  1 70 71 79  1 75]
 [ 1 65 75 70  7 76 13  1 48 64]
 [75 57 65 60  1 76 71  1 64 61]]


## Inputs

In [8]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

## LSTM CELL

In [9]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    
    def build_cell(lstm_size, keep_prob):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

## RNN Output

In [10]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        x: Input tensor
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # That is, the shape should be batch_size*num_steps rows by lstm_size columns
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

## Training loss

In [11]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per batch_size per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    return loss

## Optimizer

In [12]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

## Build the network

In [13]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN and collect the outputs
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

## Hyperparameters

In [14]:
batch_size = 100        # Sequences per batch
num_steps = 100         # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001   # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Training

In [15]:
epochs = 20
# Print losses every N interations
print_every_n = 50

# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            if (counter % print_every_n == 0):
                end = time.time()
                print('Epoch: {}/{}... '.format(e+1, epochs),
                      'Training Step: {}... '.format(counter),
                      'Training loss: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future major versions of TensorFlow will allow gradients

## Saved checkpoints

In [16]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i3960_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i3000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i3200_l512.ckpt"
all_model_checkpoint_pa

## Sampling

In [17]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [18]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [19]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints/i3960_l512.ckpt'

In [24]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="What shall we drink?")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i3960_l512.ckpt
What shall we drink?"

"Oh, though you'll see the passions of that man so miserable for her
what you've been always a living--there, anyway to think to me," he
said. "Why would not be done! With him. The position is an orce more
in my husband to set it. I'm not simply find in it or my signiegy."

"I have never disliked them in my cast is," he asked again, and
he had no significances and her hand, which, had been seen from the
standing of the stall, and would be the child that he should not
child her and sort of the dissatisfaction only, the present cart what
they were talking of their position, and all, as though to blame for him to
break the chief thing. This tone which had been so station, and so that
he had been saying that she did not advite the same thing. They were to
leave the carriage of his sister, and went out of her house, but the
streets were not seemed to hear the porer a possible same train in the
same 

In [25]:
checkpoint = 'checkpoints/i200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="What shall we drink?")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i200_l512.ckpt
What shall we drink? hot, so the
shar te the silis ho the with sate at ou to areren to the ate whe ha thar thet he serises soutint he taresson hhe woug to hererasde th sere whas tores he hes ans hes
ond ho tount or tithe the we he sere the sard sad
tint tot who whase woud to me ant he tores and ont thes he whu sed on asd to he al toun ar ofethin hor ho she whor aft as ase ofinge th sos tan on oto the hosered.""
"Wens as woud the hhan hith she war and and het to therentos wing on tate se wito ther and sating ar he heade so he te tee antis ante hing, hate ans her serad, sous, wond tha who te whas atot her hars she sithe an at oned soth she and wos san touns ta se hos ande the hhe we hime
hand sot ine he sothe sile winthe sersithis has hite the as tharesing and whe se alt thom the sosen sas it he ane an to thit as ane wot the ha tase har ar te sint hot he ande here sase soting, ate the ho sho ante hh simos one so the th 

In [26]:
checkpoint = 'checkpoints/i600_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="What shall we drink?")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i600_l512.ckpt
What shall we drink?"

The potstent, were horestly the stace, and was sanding his hould," the toret, and
and were assoment, and thene the
prasing his with
she was sanking a thim, her his sacing, when so the was seet, and so did not, when she was to tink one to ally
the
tranger of a tine with that in's homenes wat the wise her was the pracking to that
would betare to the posticurt of the
posticar, and a tomporition what her him, and
sain alladsed. He wasted a santed and this hensel for athents. Afle he was the caring as the cronding
thought has sand and that see him but the she wis stel throed he cunder at
they sookel, and that the more, and ant shim she had not to see had the sould.

"Well, who was to dearest, but in the conling of the some whe could, said the
possion were the croms of
the comeser to he had tely as aloved that in the camants at the when the pecting, bethore, at a conesing of the sein wonked hurden tha

In [27]:
checkpoint = 'checkpoints/i1200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="What shall we drink?")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i1200_l512.ckpt
What shall we drink? He deen a
manther.

"Why won't though anyone shake you to stand it walking of him, and the crase of
the
prominion the caunes is it!" said Stepan Arkadyevitch with her hands.

He was stopling to the halls of her feething and say that they head the streng
and say any all she had been discaped to she was new she was the plain of
he were sating in his said, but it were into the streage.

"Yes, I say she was not being at the seetile to all me," said Veslovsky, and so was
alraid. "I am not it was tern as sincle,.."

"We have steakn the women's a stread in a lughity," and he was not terring of
his house, basking once a shall, and sening her he had say that the
still of all, and how the princicly the poremater, but with a same and
criad of simprese on his friess, as this shill taiking at her the sentered
thinged him of all the distorse to have been that her would he was
toll him at the senter on the ment

In [28]:
checkpoint = 'checkpoints/i2400_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="What shall we drink?")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i2400_l512.ckpt
What shall we drink? The
servants' high wife in her sarrow too to so that he cared to him
so importented a day and her hair and shouthed--to all the clurglis
of her father's face.

"I won't bried to the prince would have seen them.... What a shile of
any memblication of him. It all this is this thoughts, is though the
prance, and his bouter and how he had this meaning to surd a
little compasience and thoughts, there, all anything. All the commed
in the middle of me that have so much to set it.. We as to
be discaped about her?"

"Not so?"

The chine, his was at him to answer,. And he
had as in those minuse he had not
thought of his face, and would have then her that she was
stepping, and so it are already. And that if he could not have the
sense of his wife and at once as if it had not been been asked at
the mencal of her for asleep, and this can interested the letter
and her face were any miles, and with his sisher i

In [29]:
checkpoint = 'checkpoints/i3000_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="What shall we drink?")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i3000_l512.ckpt
What shall we drink? He was and
had a man, her husband stanced at Levin, a side of the point, and the
strange one of her hands of the princess sat down at him. "There was
nothing it all the same times there was, but instead of madiar
of the sick man to told him to see him, and heard that sensation is
offening a divorce, his best saying the more shooting. She was
already doctor a lower of humiliation. And the mind he should go,
thinking of the carreage, but her life had succeeded in the moment of
his shooting and hinking of his carriage, when the sound of the carriages
he could not help as she was not tenderny. She drove out after his head and
trying to say and tenderly. "Why is the camperting of all?" Alexey
Alexandrovitch was still mare utterly unangay fingers.

"We are straight in the passed!" he thought, "the same tenderness
are not of this mumbling feeling. What didn't some doctors here a little
moment a most man

In [30]:
checkpoint = 'checkpoints/i3600_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="What shall we drink?")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i3600_l512.ckpt
What shall we drink?"

"And you're not silent," said Veslovsky, and after simply in his
state, shameful friends and something will have been talking to his
wife.

"What does they say the same, you know who doves were the family, I saw
him..."

"Oh, I haven't the mire that's the same to see you, I'm not so always
better, and that I could have a money with the that was tell me the
messones, an interrupting these times that I don't say this..."

"When she does not get out things," he said; sat somewhere.

Alexey Alexandrovitch--the parent he had not been at his heart which had
sund the chillren, sent him to an easy official parts and himself,
angiral that had always do the measor and all the whole sighe of the
chird of such an acquaintance which he had been the point of his opner
things, and all the princess started along the means, but the point
of high the carriage, and the same time his stopping son with her figure
w

In [31]:
checkpoint = 'checkpoints/i3800_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="What shall we drink?")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i3800_l512.ckpt
What shall we drink? His
course had nothing to say, but she had been consented. She would
hard to him to take, he sup to attractive tense, and his head and
common crack to be angres, as soon as supposed in act and self-passed
head to the conversations that the conversation in special poritors,
and there would be to see it, and that he has any divers when it
seemed to the cherk, and his condition was now it all at her times to
home to him that it would not talk to the stear to the most lost in
women arrung, and he had no deliberably time and take off them, and she
had been creared to him when he was too than ever. This work had noticed
it, though he could not come to sudden her, he saw that he was standing
at once, but she was not all the constroctod of the conversation. The
creets of the secret conspicution of the corner of the man who had
said, was to him, took it on an acquaintance. After his his study in
the house

In [32]:
checkpoint = 'checkpoints/i3960_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="What shall we drink?")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i3960_l512.ckpt
What shall we drink?"

"Yes, that she's still that I don't she think I am. But how say If you
can't help moved away. Alexey Alexandrovitch then I were sitting to
be, what was it, but I don't know. I shall go the passegular comparions of
an arm another. We wouldn't talk about the sacress one things. I want to
think of your husband that I can't have any more
delathing," he said, with a sister was his face, and he had
been delighted, but there were told his mother. But a carriage he heard
a low, they shall so saying what he was being able to be indeed that
to be answering his father about the same drawing room.

A few worried the cries of these plinss, which he had sent a man of the
fertority--a second, say whether they heard her. And that it would be a
might, that he can say to him to be a sick and sands of her arm, and
his face said himself, and with a crossing household was a man,
tone in the country. To the same tim